### Database Structure

#### Create normalized tables from cleaned data

This section creates a relational database structure by:

- Creating dimension tables: `producto`, `tipo_producto`, `cliente`, `tipo_cliente`, `tipo_venta`.
- Creating fact table: `ventas` with foreign keys to dimension tables.
- Ensuring data normalization and referential integrity.
- Saving tables to the curated data folder.

In [1]:
from pandas import DataFrame, read_parquet
from pathlib import Path

# DATA LOADING
df = read_parquet("../data/staging/Sales.parquet")

# DIMENSION TABLES

# Table: Cities
df_cities = DataFrame({"city": df["ciudad"].unique()})
df_cities = df_cities.reset_index().rename(columns={"index": "id_city"})
df_cities["id_city"] += 1
print(f"\nCities: {len(df_cities)} rows")
display(df_cities.head())

# Table: Customer Types
df_types_customers = DataFrame({"customer_type": df["tipo_cliente"].unique()})
df_types_customers = df_types_customers.reset_index().rename(
    columns={"index": "id_type_customer"}
)
df_types_customers["id_type_customer"] += 1
print(f"\nCustomer Types: {len(df_types_customers)} rows")
display(df_types_customers.head())

# Table: Customers (combines city + customer type)
df_customers = df[["ciudad", "tipo_cliente"]].drop_duplicates().reset_index(drop=True)
df_customers = df_customers.merge(
    df_cities, left_on="ciudad", right_on="city", how="left"
)
df_customers = df_customers.merge(
    df_types_customers, left_on="tipo_cliente", right_on="customer_type", how="left"
)
df_customers = (
    df_customers[["id_city", "id_type_customer"]]
    .reset_index()
    .rename(columns={"index": "id_customer"})
)
df_customers["id_customer"] += 1
print(f"\nCustomers: {len(df_customers)} rows")
display(df_customers.head())

# Table: Product Types
df_types_products = DataFrame({"product_type": df["tipo_producto"].unique()})
df_types_products = df_types_products.reset_index().rename(
    columns={"index": "id_type_product"}
)
df_types_products["id_type_product"] += 1
print(f"\nProduct Types: {len(df_types_products)} rows")
display(df_types_products.head())

# Table: Products
df_products = df[["producto", "tipo_producto"]].drop_duplicates().reset_index(drop=True)
df_products = df_products.merge(
    df_types_products, left_on="tipo_producto", right_on="product_type", how="left"
)
df_products = df_products[["producto", "id_type_product"]].rename(
    columns={"producto": "product_name"}
)
df_products = df_products.reset_index().rename(columns={"index": "id_product"})
df_products["id_product"] += 1
df_products = df_products[["id_product", "product_name", "id_type_product"]]
print(f"\nProducts: {len(df_products)} rows")
display(df_products.head())

# Table: Sale Types
df_types_sales = DataFrame({"sale_type": df["tipo_venta"].unique()})
df_types_sales = df_types_sales.reset_index().rename(columns={"index": "id_type_sale"})
df_types_sales["id_type_sale"] += 1
print(f"\nSale Types: {len(df_types_sales)} rows")
display(df_types_sales.head())

# FACT TABLE: SALES

# Build sales table with foreign keys
df_sales = df.copy()

# Merge with dimension tables to get IDs
df_sales = df_sales.merge(
    df_products, left_on="producto", right_on="product_name", how="left"
)
df_sales = df_sales.merge(
    df_types_sales, left_on="tipo_venta", right_on="sale_type", how="left"
)

# Create temporary helper to get customer IDs
df_customer_lookup = (
    df[["ciudad", "tipo_cliente"]].drop_duplicates().reset_index(drop=True)
)

df_customer_lookup = df_customer_lookup.merge(
    df_customers.merge(df_cities, on="id_city").merge(
        df_types_customers, on="id_type_customer"
    ),
    left_on=["ciudad", "tipo_cliente"],
    right_on=["city", "customer_type"],
    how="left",
)[["ciudad", "tipo_cliente", "id_customer"]]

df_sales = df_sales.merge(df_customer_lookup, on=["ciudad", "tipo_cliente"], how="left")

# Select and rename final columns
df_sales = df_sales[
    [
        "fecha",
        "id_product",
        "id_type_sale",
        "id_customer",
        "cantidad",
        "precio_unitario",
        "descuento",
        "costo_envio",
    ]
].rename(
    columns={
        "fecha": "date",
        "cantidad": "quantity",
        "precio_unitario": "unit_price",
        "descuento": "discount",
        "costo_envio": "shipping_cost",
    }
)

# Add primary key
df_sales = df_sales.reset_index().rename(columns={"index": "id_sale"})
df_sales["id_sale"] += 1
df_sales = df_sales[
    [
        "id_sale",
        "date",
        "id_product",
        "id_type_sale",
        "id_customer",
        "quantity",
        "unit_price",
        "discount",
        "shipping_cost",
    ]
]

print(f"\nSales: {len(df_sales)} rows")
display(df_sales.head())

# SAVE TABLES
curated_path = Path("../data/curated/")
curated_path.mkdir(parents=True, exist_ok=True)

# Dictionary with all tables
tables = {
    "cities": df_cities,
    "customer_types": df_types_customers,
    "customers": df_customers,
    "product_types": df_types_products,
    "products": df_products,
    "sale_types": df_types_sales,
    "sales": df_sales,
}

# Save each table to Parquet
curated_path.mkdir(parents=True, exist_ok=True)
print(f"\n{'=' * 10} Saving Tables {'=' * 10}")
for table_name, table_df in tables.items():
    file_path = curated_path / f"{table_name}.parquet"
    table_df.to_parquet(file_path, engine="pyarrow", compression="zstd", index=False)
    print(f"✓ {table_name}: {len(table_df)} rows → {file_path.name}")



Cities: 33 rows


,id_city,city
0,1,Santiago
1,2,Córdoba
2,3,Barranquilla
3,4,New york
4,5,Madrid



Customer Types: 4 rows


,id_type_customer,customer_type
0,1,Minorista
1,2,Gobierno
2,3,Mayorista
3,4,Corporativo



Customers: 132 rows


...


Sale Types: 4 rows


,id_type_sale,sale_type
0,1,Online
1,2,Distribuidor
2,3,Tienda física
3,4,Call center



Sales: 7431930 rows


,id_sale,date,id_product,id_type_sale,id_customer,quantity,unit_price,discount,shipping_cost
0,1,2025-10-30,1,1,1,2,3681.0,0.2,5889.0
1,2,2025-10-30,6,1,1,2,3681.0,0.2,5889.0
2,3,2025-10-30,26,1,1,2,3681.0,0.2,5889.0
3,4,2025-10-30,32,1,1,2,3681.0,0.2,5889.0
4,5,2025-10-30,42,1,1,2,3681.0,0.2,5889.0



========== Saving Tables ==========
✓ cities: 33 rows → cities.parquet
✓ customer_types: 4 rows → customer_types.parquet
✓ customers: 132 rows → customers.parquet
✓ product_types: 6 rows → product_types.parquet
✓ products: 72 rows → products.parquet
✓ sale_types: 4 rows → sale_types.parquet
✓ sales: 7431930 rows → sales.parquet
